***

*Course:* [Math 535](https://people.math.wisc.edu/~roch/mmids/) - Mathematical Methods in Data Science (MMiDS)  
*Chapter:* 6-Optimization theory and algorithms   
*Author:* [Sebastien Roch](https://people.math.wisc.edu/~roch/), Department of Mathematics, University of Wisconsin-Madison  
*Updated:* Feb 9, 2024   
*Copyright:* &copy; 2024 Sebastien Roch

***

In [ ]:
# You will need the files:
#     * mmids.py
#     * advertising.csv 
# from https://github.com/MMiDS-textbook/MMiDS-textbook.github.io/tree/main/utils
#
# IF RUNNING ON GOOGLE COLAB (RECOMMENDED):
# "Upload to session storage" from the Files tab on the left
# Alternative instructions: https://colab.research.google.com/notebooks/io.ipynb

In [ ]:
# PYTHON 3
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import mmids
seed = 535
rng = np.random.default_rng(seed)
import warnings
warnings.filterwarnings('ignore')

## Motivating example:  deciphering handwriting

We now turn to classification.

Quoting [Wikipedia](https://en.wikipedia.org/wiki/Statistical_classification):

> In machine learning and statistics, classification is the problem of identifying to which of a set of categories (sub-populations) a new observation belongs, on the basis of a training set of data containing observations (or instances) whose category membership is known. Examples are assigning a given email to the "spam" or "non-spam" class, and assigning a diagnosis to a given patient based on observed characteristics of the patient (sex, blood pressure, presence or absence of certain symptoms, etc.). Classification is an example of pattern recognition. In the terminology of machine learning, classification is considered an instance of supervised learning, i.e., learning where a training set of correctly identified observations is available.

We will illustrate this problem on the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset. Quoting [Wikipedia](https://en.wikipedia.org/wiki/MNIST_database) again:

> The MNIST database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems. The database is also widely used for training and testing in the field of machine learning. It was created by "re-mixing" the samples from NIST's original datasets. The creators felt that since NIST's training dataset was taken from American Census Bureau employees, while the testing dataset was taken from American high school students, it was not well-suited for machine learning experiments. Furthermore, the black and white images from NIST were normalized to fit into a 28x28 pixel bounding box and anti-aliased, which introduced grayscale levels. The MNIST database contains 60,000 training images and 10,000 testing images. Half of the training set and half of the test set were taken from NIST's training dataset, while the other half of the training set and the other half of the test set were taken from NIST's testing dataset.

**Figure:** MNIST sample images ([Source](https://commons.wikimedia.org/wiki/File:MnistExamples.png))

![MNIST sample images](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

$\bowtie$

We first load the data and convert it to an appropriate matrix representation. The data can be accessed with [`torchvision.datasets.MNIST`](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html).

In [ ]:
import torch
from torchvision import datasets, transforms

In [ ]:
# Download and load the MNIST dataset
mnist = datasets.MNIST(root='./data', 
                       train=True, 
                       download=True, 
                       transform=transforms.ToTensor())

# Convert the dataset to a PyTorch DataLoader
train_loader = torch.utils.data.DataLoader(mnist, 
                                           batch_size=len(mnist), 
                                           shuffle=False)

# Extract images and labels from the DataLoader
imgs, labels = next(iter(train_loader))
imgs = imgs.squeeze().numpy()
labels = labels.numpy()

The [`squeeze()`](https://pytorch.org/docs/stable/generated/torch.Tensor.squeeze.html) above removes the color dimension in the image, which is grayscale. The [`numpy()`](https://pytorch.org/docs/stable/generated/torch.Tensor.numpy.html) converts the PyTorch tensors into Numpy arrays. See [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for details on the data loading. We will say more about PyTorch later in this chapter.

For example, the first image and its label are:

In [ ]:
plt.figure()
plt.imshow(imgs[0])
plt.show()

In [ ]:
labels[0]

For now, we look at a subset of the samples: the 0's and 1's.

In [ ]:
# Filter out images with labels 0 and 1
mask = (labels == 0) | (labels == 1)
imgs01 = imgs[mask]
labels01 = labels[mask]

In this new dataset, the first sample is:

In [ ]:
plt.figure()
plt.imshow(imgs01[0])
plt.show()

In [ ]:
labels01[0]

Next, we transform the images into vectors. For this we use the [`reshape()`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html) function, which changes the dimensions of an array without changing its data. Here the first dimension, which runs across the samples, remains of the same length `len(imgs01)`. The `-1` is understood to be whatever is needed to "fit" the remaining dimensions, here $28 \times 28 = 784$. In other words, we are effectively "flattening" each $28 \times 28$ image into a single vector of length $784$. 

In [ ]:
imgs01.shape

In [ ]:
X = imgs01.reshape(len(imgs01), -1)

In [ ]:
X.shape

In [ ]:
y = labels01

The input data is now of the form $\{(\mathbf{x}_i, y_i) : i=1,\ldots, n\}$ where $\mathbf{x}_i \in \mathbb{R}^d$ are the features and $y_i \in \{0,1\}$ is the label. Above we use the matrix representation $X \in \mathbb{R}^{d \times n}$ with columns $\mathbf{x}_i$, $i = 1,\ldots, n$ and $\mathbf{y} = (y_1, \ldots, y_n)^T \in \{0,1\}^n$. 

Our goal: 

> learn a classifier from the examples $\{(\mathbf{x}_i, y_i) : i=1,\ldots, n\}$, that is, a function $\hat{f} : \mathbb{R}^d \to \mathbb{R}$ such that $\hat{f}(\mathbf{x}_i) \approx y_i$.

We may want to enforce that the output is in $\{0,1\}$ as well. This problem is referred to as [binary classification](https://en.wikipedia.org/wiki/Binary_classification).

## Background: review of differentiable functions of several variables and introduction to automatic differentiation

### Brief introduction to automatic differentiation

We illustrate the use of [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) to compute gradients. 

Quoting [Wikipedia](https://en.wikipedia.org/wiki/Automatic_differentiation):

> In mathematics and computer algebra, automatic differentiation (AD), also called algorithmic differentiation or computational differentiation, is a set of techniques to numerically evaluate the derivative of a function specified by a computer program. AD exploits the fact that every computer program, no matter how complicated, executes a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (exp, log, sin, cos, etc.). By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to working precision, and using at most a small constant factor more arithmetic operations than the original program. Automatic differentiation is distinct from symbolic differentiation and numerical differentiation (the method of finite differences). Symbolic differentiation can lead to inefficient code and faces the difficulty of converting a computer program into a single expression, while numerical differentiation can introduce round-off errors in the discretization process and cancellation.

We will use [PyTorch](https://pytorch.org/tutorials/). It uses [tensors](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html), which in many ways behave similarly to Numpy arrays. See [here](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html) for a quick introduction. Here is an example. We first initialize the tensors. Here each corresponds to a single real variable. With the option [`requires_grad=True`](https://pytorch.org/docs/stable/generated/torch.Tensor.requires_grad.html#torch.Tensor.requires_grad), we indicate that these are variables with respect to which a gradient will be taken later. We initialize the tensors where the derivatives will be computed.

In [ ]:
# Initialize variables
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)

The function [`.backward()`](https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html) computes the gradient using backpropagation, to which we will return later. The partial derivatives are accessed with [`.grad`](https://pytorch.org/docs/stable/generated/torch.Tensor.grad.html).

In [ ]:
# Perform automatic differentiation
f = 3 * x**2 + torch.exp(x) + y
f.backward()  # Compute gradients

In [ ]:
# Print gradients
print(x.grad)  # df/dx
print(y.grad)  # df/dy

The input parameters can also be vectors, which allows to consider function of large numbers of variables. 

In [ ]:
# New variables for the second example
z = torch.tensor([1., 2., 3.], requires_grad=True)

In [ ]:
# Perform automatic differentiation
g = torch.sum(z**2)
g.backward()  # Compute gradients

In [ ]:
# Print gradient
print(z.grad)  # gradient is (2 z_1, 2 z_2, 2 z_3)

Here is another typical example in a data science context.

In [ ]:
# Variables for the third example
X = torch.randn(3, 2)  # Random dataset (features)
y = torch.tensor([[1., 0., 1.]])  # Dataset (labels)
theta = torch.ones(2, 1, requires_grad=True)  # Parameter assignment

In [ ]:
# Perform automatic differentiation
predict = X @ theta  # Classifier with parameter vector theta
loss = torch.sum((predict - y)**2)  # Loss function
loss.backward()  # Compute gradients

In [ ]:
# Print gradient
print(theta.grad)  # gradient of loss

$\newcommand{\bSigma}{\boldsymbol{\Sigma}}$ $\newcommand{\bmu}{\boldsymbol{\mu}}$

## Optimality conditions and convexity

**EXAMPLE:** Consider $f(x) = e^x$. Then $f'(x) = f''(x) = e^x$. Suppose we are interested in approximating $f$ in the interval $[0,1]$. We take $a=0$ and $b=1$ in *Taylor's Theorem*. The linear term is 

$$
f(a) + (x-a) f'(a) = 1 + x e^0 = 1 + x.
$$

Then for any $x \in [0,1]$

$$
f(x) = 1 + x + \frac{1}{2}x^2 e^{\xi_x}
$$

where $\xi_x \in (0,1)$ depends on $x$. We get a uniform bound on the error over $[0,1]$ by replacing $\xi_x$ with its worst possible value over $[0,1]$ 

$$
|f(x) - (1+x)| \leq \frac{1}{2}x^2 e^{\xi_x} \leq \frac{e}{2} x^2.
$$

In [ ]:
x = np.linspace(0,1,100)
y = np.exp(x)
taylor = 1 + x
err = (np.exp(1)/2) * x**2

In [ ]:
plt.plot(x,y,label='f')
plt.plot(x,taylor,label='taylor')
plt.legend()
plt.show()

If we plot the upper and lower bounds, we see that $f$ indeed falls within them.

In [ ]:
plt.plot(x,y,label='f')
plt.plot(x,taylor,label='taylor')
plt.plot(x,taylor-err,linestyle=':',color='green',label='lower')
plt.plot(x,taylor+err,linestyle='--',color='green',label='upper')
plt.legend()
plt.show()

$\lhd$

**EXAMPLE:** Let $f(x) = x^3$. Then $f'(x) = 3 x^2$ and $f''(x) = 6 x$ so that $f'(0) = 0$ and $f''(0) \geq 0$. Hence $x=0$ is a stationary point. But $x=0$ is not a local minimizer. Indeed $f(0) = 0$ but, for any $\delta > 0$, $f(-\delta) < 0$.

In [ ]:
x = np.linspace(-2,2,100)
y = x**3

In [ ]:
plt.plot(x,y)
plt.ylim(-5,5)
plt.show()

$\lhd$

## Gradient descent and its convergence analysis

**NUMERICAL CORNER:** We implement gradient descent in Python. We assume that a function `f` and its gradient `grad_f` are provided. We first code the basic steepest descent step with a step size $\alpha =$ `alpha`.

In [ ]:
def desc_update(grad_f, x, alpha):
    return x - alpha*grad_f(x)

In [ ]:
def gd(f, grad_f, x0, alpha=1e-3, niters=int(1e6)):
    
    xk = x0
    for _ in range(niters):
        xk = desc_update(grad_f, xk, alpha)

    return xk, f(xk)

We illustrate on a simple example.

In [ ]:
def f(x): 
    return (x-1)**2 + 10

In [ ]:
xgrid = np.linspace(-5,5,100)
plt.plot(xgrid, f(xgrid))
plt.show()

In [ ]:
def grad_f(x):
    return 2*(x-1)

In [ ]:
gd(f, grad_f, 0)

We found a global minmizer in this case.

The next example shows that a different local minimizer may be reached depending on the starting point.

In [ ]:
def f(x): 
    return 4 * (x-1)**2 * (x+1)**2 - 2*(x-1)

In [ ]:
xgrid = np.linspace(-2,2,100)
plt.plot(xgrid, f(xgrid), label='f')
plt.ylim((-1,10))
plt.legend()
plt.show()

In [ ]:
def grad_f(x): 
    return 8 * (x-1) * (x+1)**2 + 8 * (x-1)**2 * (x+1) - 2

In [ ]:
xgrid = np.linspace(-2,2,100)
plt.plot(xgrid, f(xgrid), label='f')
plt.plot(xgrid, grad_f(xgrid), label='grad_f')
plt.ylim((-10,10))
plt.legend()
plt.show()

In [ ]:
gd(f, grad_f, 0)

In [ ]:
gd(f, grad_f, -2)

In the final example, we end up at a stationary point that is not a local minimizer. Here both the first and second derivatives are zero. This is known as a [saddle point](https://en.wikipedia.org/wiki/Saddle_point).

In [ ]:
def f(x):
    return x**3

In [ ]:
xgrid = np.linspace(-2,2,100)
plt.plot(xgrid, f(xgrid), label='f')
plt.ylim((-10,10))
plt.legend()
plt.show()

In [ ]:
def grad_f(x):
    return 3 * x**2

In [ ]:
xgrid = np.linspace(-2,2,100)
plt.plot(xgrid, f(xgrid), label='f')
plt.plot(xgrid, grad_f(xgrid), label='grad_f')
plt.ylim((-10,10))
plt.legend()
plt.show()

In [ ]:
gd(f, grad_f, 2)

In [ ]:
gd(f, grad_f, -2, niters=100)

$\unlhd$

**NUMERICAL CORNER:** We give a numerical example using a special case of logistic regression. We illustrate it on a random dataset. The functions $\hat{f}$, $\mathcal{L}$ and $\frac{\partial}{\partial x}\mathcal{L}$ are defined next.

In [ ]:
def fhat(x,a):
    return 1 / ( 1 + np.exp(-np.outer(x,a)) )

In [ ]:
def loss(x,a,b): 
    return np.mean(-b*np.log(fhat(x,a)) - (1 - b)*np.log(1 - fhat(x,a)), axis=1)

In [ ]:
def grad(x,a,b):
    return -np.mean((b - fhat(x,a))*a, axis=1)

In [ ]:
n = 10000
a = 2*rng.uniform(0,1,n) - 1
b = rng.integers(2, size=n)
x = np.linspace(-1,1,100)

In [ ]:
plt.plot(x, loss(x,a,b), label='loss')
plt.legend()
plt.show()

We plot next the upper and lower bounds in the *Quadratic Bound for Smooth Functions* around $x = x_0$. It turns out we can take $L=1$. Observe that minimizing the upper quadratic bound leads to a decrease in $\mathcal{L}$.

In [ ]:
x0 = -0.3
x = np.linspace(x0-0.05,x0+0.05,100)
upper = loss(x0,a,b) + (x - x0)*grad(x0,a,b) + (1/2)*(x - x0)**2 # upper approximation
lower = loss(x0,a,b) + (x - x0)*grad(x0,a,b) - (1/2)*(x - x0)**2 # lower approximation

In [ ]:
plt.plot(x, loss(x,a,b), label='loss')
plt.plot(x, upper, label='upper')
plt.plot(x, lower, label='lower')
plt.legend()
plt.show()

$\unlhd$

**NUMERICAL CORNER:** We revisit our first simple single-variable example.

In [ ]:
def f(x): 
    return (x-1)**2 + 10

In [ ]:
xgrid = np.linspace(-5,5,100)
plt.plot(xgrid, f(xgrid))
plt.show()

Recall that the first derivative is:

In [ ]:
def grad_f(x):
    return 2*(x-1)

So the second derivative is $f''(x) = 2$. Hence, this $f$ is $L$-smooth and $m$-strongly convex with $L = m = 2$. The theory we developed suggests taking step size $\alpha_t = \alpha = 1/L = 1/2$. It also implies that

$$
f(x^1) - f(x^*)
\leq \left(1 - \frac{m}{L}\right) [f(x^0) - f(x^*)]
= 0.
$$

We converge in one step! And that holds for any starting point $x^0$.

Let's try this!

In [ ]:
gd(f, grad_f, 0, alpha=0.5, niters=1)

Let's try a different starting point.

In [ ]:
gd(f, grad_f, 100, alpha=0.5, niters=1)

$\unlhd$

### Implementing gradient descent in PyTorch

Rather than explicitly specifying the gradient function, we could use PyTorch to compute it automatically. This is done next. Note that the descent update is done within [`with torch.no_grad()`](https://pytorch.org/docs/stable/generated/torch.no_grad.html), which ensures that the update operation itself is not tracked for gradient computation. Here the input `x0` as well as the output `xk.numpy(force=True)` are Numpy arrays. The function [`numpy()`](https://pytorch.org/docs/stable/generated/torch.Tensor.numpy.html) converts a PyTorch tensor to a Numpy array (see the documentation for an explanation of the `force=True` option).

In [ ]:
def gd_with_ad(f, x0, alpha=1e-3, niters=int(1e6)):
    xk = torch.tensor(x0, 
                      requires_grad=True, 
                      dtype=torch.float)
    
    for _ in range(niters):
        # Compute the function value and its gradient
        value = f(xk)
        value.backward()

        # Perform a gradient descent step
        with torch.no_grad():  # Temporarily set all requires_grad flags to False
            xk -= alpha * xk.grad

        # Zero the gradients for the next iteration
        xk.grad.zero_()

    return xk.numpy(force=True), f(xk).item()

We revisit a previous example.

In [ ]:
def f(x):
    return x**3

In [ ]:
xgrid = np.linspace(-2,2,100)
plt.plot(xgrid, f(xgrid), label='f')
plt.ylim((-10,10))
plt.legend()
plt.show()

In [ ]:
gd_with_ad(f, 2, niters=int(1e4))

In [ ]:
gd_with_ad(f, -2, niters=100)

$\unlhd$

## Backpropagation and application to neural networks

**The `Advertising` dataset and the least-squares solution** We return to the `Advertising` dataset.

In [ ]:
df = pd.read_csv('advertising.csv')
df.head()

In [ ]:
n = len(df.index)
print(n)

We first compute the solution using the least-squares approach we detailed previously. We use [`numpy.column_stack`](https://numpy.org/doc/stable/reference/generated/numpy.column_stack.html#numpy.column_stack) to add a column of ones to the feature vectors.

In [ ]:
TV = df['TV'].to_numpy()
radio = df['radio'].to_numpy()
newspaper = df['newspaper'].to_numpy()
sales = df['sales'].to_numpy()
features = np.stack((TV, radio, newspaper), axis=-1)
A = np.column_stack((np.ones(n), features))

In [ ]:
coeff = mmids.ls_by_qr(A, sales)
print(coeff)

In [ ]:
np.mean((A @ coeff - sales)**2)

**Solving the problem using PyTorch** We will be using PyTorch to implement the previous method. We first convert the data into PyTorch tensors. We then use [`torch.utils.data.TensorDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset) to create the dataset. Finally, [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) provides the utilities to load the data in batches for training. We take mini-batches of size `BATCH_SIZE = 64` and we apply a random permutation of the samples on every pass (with the option `shuffle=True`).

In [ ]:
# Convert data to PyTorch tensors
features_tensor = torch.tensor(features, 
                               dtype=torch.float32)
sales_tensor = torch.tensor(sales, 
                            dtype=torch.float32).view(-1, 1)

In [ ]:
# Create a dataset and dataloader for training
BATCH_SIZE = 64
train_dataset = TensorDataset(features_tensor, sales_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

Now we construct our model. It is simply an affine map from $\mathbb{R}^3$ to $\mathbb{R}$. Note that there is no need to pre-process the inputs by adding $1$s. A constant term (or "bias variable") is automatically added by PyTorch (unless one chooses the option [`bias=False`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)).

In [ ]:
# Define the model using nn.Sequential
model = nn.Sequential(
    nn.Linear(3, 1)  # 3 input features, 1 output value
)

Finally, we are ready to run an optimization method of our choice on the loss function, which are specified next. There are many [optimizers](https://pytorch.org/docs/stable/optim.html#algorithms) available. (See this [post](https://hackernoon.com/demystifying-different-variants-of-gradient-descent-optimization-algorithm-19ae9ba2e9bc) for a brief explanation of many common optimizers.) Here we use SGD as the optimizer. And the loss function is the MSE. A quick tutorial is [here](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html).

Choosing the right number of passes (i.e. epochs) through the data requires some experimenting. Here $10^4$ suffices. But in the interest of time, we will run it only for $100$ epochs. As you will see from the results, this is not quite enough. On each pass, we compute the output of the current model, use `backward()` to obtain the gradient, and then perform a descent update with `step()`. We also have to reset the gradients first (otherwise they add up by default). 

In [ ]:
# Compile the model: define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [ ]:
# Train the model
epochs = 100
for epoch in range(epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

The final parameters and loss are:

In [ ]:
# Get and print the model weights and bias
weights = model[0].weight.detach().numpy()
bias = model[0].bias.detach().numpy()
print("Weights:", weights)
print("Bias:", bias)

In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    total_loss = 0
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()
        
    print(f"Mean Squared Error on Training Set: {total_loss / len(train_loader)}")

**MNIST dataset** We will use the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset introduced earlier in the chapter. This example is inspired by [these](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) [tutorial](https://www.tensorflow.org/tutorials/keras/classification).

**Figure:** MNIST sample images ([Source](https://commons.wikimedia.org/wiki/File:MnistExamples.png))

![MNIST sample images](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

$\bowtie$

We first load the data. As before, the training dataset is a tensor -- think matrix with $3$ indices. One index runs through the $60,000$ training images, while the other two indices run through the horizontal and vertical pixel axes of each image. Here each image is $28 \times 28$. The training labels are between $0$ and $9$.

In [ ]:
# Load and normalize the MNIST dataset
train_dataset = datasets.MNIST(root='./data', 
                               train=True, 
                               download=True, 
                               transform=transforms.ToTensor())

test_dataset = datasets.MNIST(root='./data', 
                              train=False, 
                              download=True, 
                              transform=transforms.ToTensor())

In [ ]:
BATCH_SIZE = 32
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False)

**Implementation** We implement multinomial logistic regression to learn a classifier for the MNIST data. We first check for the availability of GPUs.

In [ ]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In PyTorch, composition of functions can be achieved with [`torch.nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html). Our model is:

In [ ]:
# Define the model using nn.Sequential and move it to the device (GPU if available)
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28 * 28, 10)
).to(device)

The [`torch.nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer turns each input image into a vector of size $784$ (where $784 = 28^2$ is the number of pixels in each image). The final output is $10$-dimensional.

Here we use the [`torch.optim.Adam`](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html) optimizer (you can try SGD, but it is slow). The loss function is the [cross-entropy](https://en.wikipedia.org/wiki/Cross_entropy), as implemented by [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

In [ ]:
# Compile the model: define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In the interest of time, we train for 3 epochs only. An epoch is one training iteration where all samples are iterated once (in a randomly shuffled order).

In [ ]:
# Training function
def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
# Training loop
epochs = 3  # Adjust the number of epochs as needed
for epoch in range(epochs):
    train(train_loader, model, loss_fn, optimizer, device)
    
    if (epoch+1) % 1 == 0:
        print(f"Epoch {epoch+1}/{epochs}")

Because of the issue of [overfitting](https://en.wikipedia.org/wiki/Overfitting), we use the *test* images to assess the performance of the final classifier. 

In [ ]:
# Evaluation function
def test(dataloader, model, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(dim=1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    accuracy = correct / size
    print(f"Test error: {(100*accuracy):>0.1f}% accuracy")

In [ ]:
# Evaluate the model
test(test_loader, model, device)

To make a prediction, we take a [`torch.nn.functional.softmax`](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html) of the output of our model. It transforms the output into a probability for each label. It is implicitly included in the cross-entropy loss, but is not actually part of the trained model. (Note that the softmax itself has no parameter.) 

As an illustration, we do this for each test image. We use [`torch.cat`](https://pytorch.org/docs/stable/generated/torch.cat.html) to concatenate a sequence of tensors into a single tensor.

In [ ]:
def predict_softmax(dataloader, model, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()  # Set the model to evaluation mode
    predictions = []
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            probabilities = F.softmax(pred, dim=1)
            predictions.append(probabilities.cpu())  # Move predictions to CPU

    return torch.cat(predictions, dim=0)

predictions = predict_softmax(test_loader, model, device).numpy()

The result for the first test image is shown below. To make a prediction, we choose the label with the highest probability.

In [ ]:
print(predictions[0])

In [ ]:
predictions[0].argmax(0)

The truth is:

In [ ]:
images, labels = next(iter(test_loader))
images = images.squeeze().numpy()
labels = labels.numpy()
labels[0]

Above, `next(iter(test_loader))` loads the first batch of test images. (See [here](https://docs.python.org/3/tutorial/classes.html#iterators) for background on iterators in Python.)

The following code, adapted from [here](https://www.tensorflow.org/tutorials/keras/classification), provides a neat vizualization of the results.

In [ ]:
class_names = ['0', '1', '2', '3', '4',
               '5', '6', '7', '8', '9']

def plot_image(predictions_array, true_label, img):
    
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel(f"{class_names[predicted_label]} {100*np.max(predictions_array):2.0f}% ({class_names[true_label]})", 
               color=color)

def plot_value_array(predictions_array, true_label):
    
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)
 
    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

Here's the first one.

In [ ]:
# Visualization code for individual image
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(predictions[i], labels[i], images[i])  
plt.subplot(1,2,2)
plot_value_array(predictions[i], labels[i]) 
plt.show()

This one is a little less clear. 

In [ ]:
# Visualization code for individual and multiple images
i = 11
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(predictions[i], labels[i], images[i])  
plt.subplot(1,2,2)
plot_value_array(predictions[i], labels[i]) 
plt.show()

This one is wrong.

In [ ]:
# Visualization code for individual and multiple images
i = 8
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(predictions[i], labels[i], images[i])  
plt.subplot(1,2,2)
plot_value_array(predictions[i], labels[i]) 
plt.show()

**Implementation** We implement a neural network in PyTorch. We use the MNIST dataset again. We have already loaded it.

We construct a three-layer model.

In [ ]:
# Define the model using nn.Sequential
model = nn.Sequential(
    nn.Flatten(),                      # Flatten the input
    nn.Linear(28 * 28, 32),            # First Linear layer with 32 nodes
    nn.Sigmoid(),                      # Sigmoid activation function
    nn.Linear(32, 10)                  # Second Linear layer with 10 nodes (output layer)
).to(device)

As we did for multinomial logistic regression, we use the Adam optimizer and the cross-entropy loss. We also monitor progress by keeping track of the accuracy on the training data.

In [ ]:
# Define the loss function and the optimizer
loss_fn = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters())

Again, we train for 3 epochs.

In [ ]:
# Training loop
epochs = 3  # Adjust the number of epochs as needed
for epoch in range(epochs):
    train(train_loader, model, loss_fn, optimizer, device)
    
    if (epoch+1) % 1 == 0:
        print(f"Epoch {epoch+1}/{epochs}")

On the test data, we get:

In [ ]:
# Evaluate the model
test(test_loader, model, device)

This is a significantly more accurate model than what we obtained using multinomial logistic regression. One can do even better using a neural network tailored for images, known as [convolutional neural networks](https://cs231n.github.io/convolutional-networks/).